In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
import matplotlib.patches as patches

#from astroquery.simbad import Simbad
#from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [31]:
# loading in the MAC now
MAC = pd.read_csv('../MAC_DR0p9.csv', sep=',')

# converting coordinates to degrees
coordconvert = SkyCoord(ra = MAC['RA1'], dec = MAC['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
MAC['RA1_deg'] = coordconvert.ra.degree
MAC['Dec1_deg'] = coordconvert.dec.degree

# reading in the ICRF3 tables from Charlot+
crf_K = pd.read_csv('../K_ICRF_coords_deg.csv')
crf_XKa = pd.read_csv('../XKa_ICRF_coords_deg.csv')
crf_SX = pd.read_csv('../SX_ICRF_coords_deg.csv')


crf_K['RA1_deg'] = crf_K['RA_deg']
crf_K['Dec1_deg'] = crf_K['Dec_deg']

crf_XKa['RA1_deg'] = crf_XKa['RA_deg']
crf_XKa['Dec1_deg'] = crf_XKa['Dec_deg']

crf_SX['RA1_deg'] = crf_SX['RA_deg']
crf_SX['Dec1_deg'] = crf_SX['Dec_deg']

In [33]:
#crf_SX

In [32]:
MAC1 = MAC[(MAC['RA2']=="-99") | (MAC['RA2']=="RELATIVE")]
MAC2 = MAC[(MAC['RA2']!="-99") & (MAC['RA2']!="RELATIVE")]

# this is a note to ryan that we need to go back and fix the relative note about coordinates in that one row

coordconvert = SkyCoord(ra = MAC2['RA2'], dec = MAC2['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
MAC2['RA2_deg'] = coordconvert.ra.degree
MAC2['Dec2_deg'] = coordconvert.dec.degree

#just confirming here at the table is broken up into objects with (1) one set of coordinates and \
# (2) unique coordinates for each system
print(len(MAC),len(MAC1),len(MAC2))

# we'll be matching against the N-tuple catalogs later on in this notebook

5683 4030 1653


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/2644489122.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MAC2['RA2_deg'] = coordconvert.ra.degree
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/2644489122.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MAC2['Dec2_deg'] = coordconvert.dec.degree


In [14]:
# setting up matching function
def match_tables_fib_doubles(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_1 = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_1 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    # Now we need to match RA1 and Dec1 of t1 to RA2 and Dec2 of t2 in case the pairs are mixeup between tables
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA2_deg']*u.degree, dec=t2['Dec2_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1_2, idx2_2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1_2)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2_2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_2 = pd.concat([(t1.iloc[idx1_2]),(t2.iloc[idx2_2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_2 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    return tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2


def match_tables_fib_singles(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    return tunique, tmatches, idx1, idx2


# Focusing first on the objects with two sets of coordinates in the Big MAC archive

In [29]:
# here we're matching against the MAC targets that have only a single set of coordinates for ICRF K band 
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(crf_K,MAC2,1)

print(len(idx1),len(idx2),len(idx1_2),len(idx2_2)) # 1 unique match at 1'' and 10'' (though at 10'', we \
                                                   # get a match against the second AGN candidate as well)

tmatches_1

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'


1 1 0 0


,level_0,index,Unnamed: 0,Designation,Flag,RA,DEC,RA_deg,Dec_deg,RA1_deg,Dec1_deg,Table_flag,Unnamed: 0.1,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Processed Selection Method,Parsed Selection Method,Analysis Method,Processed Analysis Method,Processed Confirmation Method,Parsed Analysis Method,Parsed Confirmation Method,RA2_deg,Dec2_deg
0,1497,2708,3100,NaN,NaN,NaN,NaN,NaN,NaN,244.156487,4.992427,Table2,2708.0,Dual AGN,PKS 1614+051,Optical Narrow-Band Imaging,Radio Imaging / Optical Imaging / Optical Narr...,PKS 1614+051 QSO,3.209,spec,16:16:37.55681502,+04:59:32.7367495,Radio,VLBI,J2000,19.17,V,mag,PKS 1614+051 A,3.214,spec,16:16:37.83,04:59:37.1,J2000,Optical,MUSE,19.17,V,mag,-99.0,6.55,49.410061,-99.0,Djorgovski+1985 ; Djorgovski+1987 ; Hu+1987 ; ...,1985ApJ...299L...1D ; 1987AJ.....93.1318D ; 19...,https://doi.org/10.1086/184569 ; https://doi.o...,Possible QSO-AGN pair at similar redshifts but...,1.0,Dual AGN,Dual AGN,Dual AGN,-99,-99,1.0,-99.0,-99.0,-354.980286,Optical Narrow-Band Imaging,{'Optical': ['Narrow-Band Imaging']},Optical Narrow-Band Imaging / Radio Imaging / ...,Optical IFU Imaging / Optical IFU Spectroscopy...,Optical IFU Imaging / Optical IFU Spectroscopy...,"{'Optical': ['IFU Imaging', 'IFU Spectroscopy'...","{'Optical': ['IFU Imaging', 'IFU Spectroscopy'...",244.157625,4.993639
1,588,588,588,ICRFJ161637.5+045932,V,16:16:37.55682638,04:59:32.7364149,244.156487,4.992427,244.156487,4.992427,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# here we're matching against the MAC targets that have only a single set of coordinates for ICRF X/KA band 
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(crf_XKa,MAC2,1)

print(len(idx1),len(idx2),len(idx1_2),len(idx2_2)) # 1 unique match at 1'' (same at 10'' but we match against \
                                                   # the second AGN candidate

# note, the match here in X/Ka band is the same as the match in K band above

tmatches_1

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'


1 1 0 0


,level_0,index,Unnamed: 0,Designation,Flag,RA,DEC,RA_deg,Dec_deg,RA1_deg,Dec1_deg,Table_flag,Unnamed: 0.1,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Processed Selection Method,Parsed Selection Method,Analysis Method,Processed Analysis Method,Processed Confirmation Method,Parsed Analysis Method,Parsed Confirmation Method,RA2_deg,Dec2_deg
0,1497,2708,3100,NaN,NaN,NaN,NaN,NaN,NaN,244.156487,4.992427,Table2,2708.0,Dual AGN,PKS 1614+051,Optical Narrow-Band Imaging,Radio Imaging / Optical Imaging / Optical Narr...,PKS 1614+051 QSO,3.209,spec,16:16:37.55681502,+04:59:32.7367495,Radio,VLBI,J2000,19.17,V,mag,PKS 1614+051 A,3.214,spec,16:16:37.83,04:59:37.1,J2000,Optical,MUSE,19.17,V,mag,-99.0,6.55,49.410061,-99.0,Djorgovski+1985 ; Djorgovski+1987 ; Hu+1987 ; ...,1985ApJ...299L...1D ; 1987AJ.....93.1318D ; 19...,https://doi.org/10.1086/184569 ; https://doi.o...,Possible QSO-AGN pair at similar redshifts but...,1.0,Dual AGN,Dual AGN,Dual AGN,-99,-99,1.0,-99.0,-99.0,-354.980286,Optical Narrow-Band Imaging,{'Optical': ['Narrow-Band Imaging']},Optical Narrow-Band Imaging / Radio Imaging / ...,Optical IFU Imaging / Optical IFU Spectroscopy...,Optical IFU Imaging / Optical IFU Spectroscopy...,"{'Optical': ['IFU Imaging', 'IFU Spectroscopy'...","{'Optical': ['IFU Imaging', 'IFU Spectroscopy'...",244.157625,4.993639
1,471,471,471,ICRFJ161637.5+045932,,16:16:37.55681118,04:59:32.7361335,244.156487,4.992427,244.156487,4.992427,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# here we're matching against the MAC targets that have only a single set of coordinates for ICRF S/X band 
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(crf_SX,MAC2,10)

print(len(idx1),len(idx2),len(idx1_2),len(idx2_2)) # 3 unique match at 1''

# note, 1/3 of these matches is the same match as in the K and X/Ka bands above
tmatches_1

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'


3 3 2 2


,level_0,index,Unnamed: 0,Designation,Flag,RA,DEC,RA_deg,Dec_deg,RA1_deg,Dec1_deg,Table_flag,Unnamed: 0.1,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Processed Selection Method,Parsed Selection Method,Analysis Method,Processed Analysis Method,Processed Confirmation Method,Parsed Analysis Method,Parsed Confirmation Method,RA2_deg,Dec2_deg
0,1444,1445,1828,NaN,NaN,NaN,NaN,NaN,NaN,241.598208,54.098861,Table2,1445.0,Dual AGN Candidate,-99,Optical Imaging / Optical Colors,-99,J160623.57+540555.90,0.87600,spec,16:06:23.57,54:05:55.90,Optical,SDSS,J2000,-100.0,-100,-100,J160624.02+540601.62,0.8820,spec,16:06:24.02,54:06:01.62,J2000,Optical,SDSS,-100.0,-100,-100,-954.760738,6.951000,53.838061,-99.0,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,-99,0.5,Dual AGN Candidate,Dual AGN Candidate,Dual AGN Candidate,-99,-99,0.5,-99.0,-99.0,-954.760738,Optical Colors / Optical Imaging,"{'Optical': ['Colors', 'Imaging']}",Optical Imaging / Optical Colors / Optical Spe...,Optical Colors / Optical Imaging / Optical Spe...,-99,"{'Optical': ['Colors', 'Imaging', 'Spectroscop...",{},241.600083,54.100450
1,1531,2743,3145,NaN,NaN,NaN,NaN,NaN,NaN,40.269994,-8.255764,Table2,2743.0,Dual AGN,NGC 1052,Hard X-ray BAT Selection,Optical Spectroscopy / Optical Fiber Spectrosc...,NGC 1052,0.00498,spec,02:41:04.79849989,-08:15:20.7519527,Radio,ICRF,J2000,-99,-99,-99,NGC 1042,0.0046,spec,02:40:23.9724818544,-08:26:00.736256616,J2000,Optical,GAIA,-99,-99,-99,-99.000000,881.307408,90.575231,-99.0,Koss+2012,2012ApJ...746L..22K,https://doi.org/10.1088/2041-8205/746/2/L22,-99,1.0,Dual AGN,Dual AGN,Dual AGN,-99,-99,1.0,-99.0,-99.0,113.078351,Hard X-ray BAT Selection,"{'Hard X-ray': ['BAT Selection'], 'X-ray': ['B...",Hard X-ray BAT Selection / Optical Spectroscop...,Hard X-ray BAT Selection / Optical Fiber Spect...,Optical Fiber Spectroscopy / Optical Spectrosc...,"{'Hard X-ray': ['BAT Selection'], 'X-ray': ['B...","{'Optical': ['Fiber Spectroscopy', 'Spectrosco...",40.099885,-8.433538
2,1497,2708,3100,NaN,NaN,NaN,NaN,NaN,NaN,244.156487,4.992427,Table2,2708.0,Dual AGN,PKS 1614+051,Optical Narrow-Band Imaging,Radio Imaging / Optical Imaging / Optical Narr...,PKS 1614+051 QSO,3.20900,spec,16:16:37.55681502,+04:59:32.7367495,Radio,VLBI,J2000,19.17,V,mag,PKS 1614+051 A,3.2140,spec,16:16:37.83,04:59:37.1,J2000,Optical,MUSE,19.17,V,mag,-99.000000,6.550000,49.410061,-99.0,Djorgovski+1985 ; Djorgovski+1987 ; Hu+1987 ; ...,1985ApJ...299L...1D ; 1987AJ.....93.1318D ; 19...,https://doi.org/10.1086/184569 ; https://doi.o...,Possible QSO-AGN pair at similar redshifts but...,1.0,Dual AGN,Dual AGN,Dual AGN,-99,-99,1.0,-99.0,-99.0,-354.980286,Optical Narrow-Band Imaging,{'Optical': ['Narrow-Band Imaging']},Optical Narrow-Band Imaging / Radio Imaging / ...,Optical IFU Imaging / Optical IFU Spectroscopy...,Optical IFU Imaging / Optical IFU Spectroscopy...,"{'Optical': ['IFU Imaging', 'IFU Spectroscopy'...","{'Optical': ['IFU Imaging', 'IFU Spectroscopy'...",244.157625,4.993639
3,520,520,520,ICRFJ024104.7-081520,,02:41:04.79849989,-08:15:20.7519527,40.269994,-8.255764,40.269994,-8.255764,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3098,3098,3098,ICRFJ160623.5+540555,V,16:06:23.56595611,54:05:55.7628327,241.598191,54.098823,241.598191,54.098823,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Now we're focusing on the objects with only a single set of coordinates

In [50]:
# here we're matching against the MAC targets that have two distinct sets of coordinates and the ICRF K band
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(crf_K,MAC1,1)

print(len(idx1),len(idx2)) # 39 unique matches using 1'', 3'', 5'', and 10'' matching tolerances

tmatches

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'


39 39


,level_0,index,Unnamed: 0,Designation,Flag,RA,DEC,RA_deg,Dec_deg,RA1_deg,Dec1_deg,Table_flag,Unnamed: 0.1,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Processed Selection Method,Parsed Selection Method,Analysis Method,Processed Analysis Method,Processed Confirmation Method,Parsed Analysis Method,Parsed Confirmation Method
0,1432,3075,3539,NaN,NaN,NaN,NaN,NaN,NaN,114.530807,17.705277,Table2,3075.0,Binary AGN Candidate,0735+178,Radio Jet Precession / Radio Imaging,-99,0735+178,0.450000,spec,07:38:07.39375312,+17:42:18.9981506,Radio,ICRF,J2000,-99,-99,-99,-99,-99.000,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99.0,-9.900000e+01,-570.183843,-99.0,Britzen+2015,2015AN....336..471B,https://doi.org/10.1002/asna.201512178,This article does not appear to have any rigor...,-0.5,Binary AGN Candidate,Binary AGN Candidate,Binary AGN Candidate,-99,-99,-0.5,-99.0,-99.0,-99.000000,Radio Imaging / Radio Jet Precession,"{'Radio': ['Imaging', 'Jet Precession']}",Radio Jet Precession / Radio Imaging / Radio I...,Radio Imaging / Radio Jet Precession,-99,"{'Radio': ['Imaging', 'Jet Precession']}",{}
1,3706,5359,5831,NaN,NaN,NaN,NaN,NaN,NaN,130.198285,13.206545,Table2,5359.0,Binary SMBH Candidate,-99,Optical Spectroscopy / Fiber Optical Spectrosc...,-99,084047.58+131223.5 / 3C 207,0.680200,spec,08:40:47.58842599,+13:12:23.5637506,Radio,ICRF,J2000,-100.0,-100,-100,-99,-99.000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,671.0,3.000000e+00,21.189475,-99.0,Liu+2014 ; Krause+2019,2014ApJ...789..140L ; 2019MNRAS.482..240K,https://doi.org/10.1088/0004-637X/789/2/140 ; ...,Krause+ claim Radio Jet Precession,0.0,Binary SMBH Candidate,Binary SMBH Candidate,Binary SMBH Candidate,-99,-99,0.0,-99.0,-99.0,671.000000,Optical Fiber Spectroscopy / Optical LOS Radia...,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",Optical Spectroscopy / Fiber Optical Spectrosc...,Optical Fiber Spectroscopy / Optical LOS Radia...,-99,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",{}
2,3737,5390,5862,NaN,NaN,NaN,NaN,NaN,NaN,141.762542,39.039111,Table2,5390.0,Binary SMBH Candidate,-99,Optical Spectroscopy / Fiber Optical Spectrosc...,-99,092703.01+390220.8,0.695200,spec,09:27:03.01,+39:02:20.8,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,964.0,3.000000e+00,21.378568,-99.0,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140,,0.0,Binary SMBH Candidate,Binary SMBH Candidate,Binary SMBH Candidate,-99,-99,0.0,-99.0,-99.0,964.000000,Optical Fiber Spectroscopy / Optical LOS Radia...,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",Optical Spectroscopy / Fiber Optical Spectrosc...,Optical Fiber Spectroscopy / Optical LOS Radia...,-99,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",{}
3,3780,5433,5905,NaN,NaN,NaN,NaN,NaN,NaN,156.186667,19.205639,Table2,5433.0,Binary SMBH Candidate,-99,Optical Spectroscopy / Fiber Optical Spectrosc...,-99,102444.80+191220.3,0.827500,spec,10:24:44.80,+19:12:20.3,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,683.0,3.000000e+00,22.780073,-99.0,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140,,0.0,Binary SMBH Candidate,Binary SMBH Candidate,Binary SMBH Candidate,-99,-99,0.0,-99.0,-99.0,683.000000,Optical Fiber Spectroscopy / Optical LOS Radia...,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",Optical Spectroscopy / Fiber Optical Spectrosc...,Optical Fiber Spectroscopy / Optical LOS Radia...,-99,"{'Optical': ['Fiber Spectroscopy', 

In [53]:
# here we're matching against the MAC targets that have two distinct sets of coordinates and the ICRF X/Ka band
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(crf_XKa,MAC1,1)

print(len(idx1),len(idx2)) # 31 unique matches using 1'', 3'', 5'', and 10'' matching tolerances

tmatches


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'


31 31


,level_0,index,Unnamed: 0,Designation,Flag,RA,DEC,RA_deg,Dec_deg,RA1_deg,Dec1_deg,Table_flag,Unnamed: 0.1,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Processed Selection Method,Parsed Selection Method,Analysis Method,Processed Analysis Method,Processed Confirmation Method,Parsed Analysis Method,Parsed Confirmation Method
0,1432,3075,3539,NaN,NaN,NaN,NaN,NaN,NaN,114.530807,17.705277,Table2,3075.0,Binary AGN Candidate,0735+178,Radio Jet Precession / Radio Imaging,-99,0735+178,0.450000,spec,07:38:07.39375312,+17:42:18.9981506,Radio,ICRF,J2000,-99,-99,-99,-99,-99.000,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99.0,-9.900000e+01,-570.183843,-99.0,Britzen+2015,2015AN....336..471B,https://doi.org/10.1002/asna.201512178,This article does not appear to have any rigor...,-0.5,Binary AGN Candidate,Binary AGN Candidate,Binary AGN Candidate,-99,-99,-0.5,-99.0,-99.0,-99.000000,Radio Imaging / Radio Jet Precession,"{'Radio': ['Imaging', 'Jet Precession']}",Radio Jet Precession / Radio Imaging / Radio I...,Radio Imaging / Radio Jet Precession,-99,"{'Radio': ['Imaging', 'Jet Precession']}",{}
1,3706,5359,5831,NaN,NaN,NaN,NaN,NaN,NaN,130.198285,13.206545,Table2,5359.0,Binary SMBH Candidate,-99,Optical Spectroscopy / Fiber Optical Spectrosc...,-99,084047.58+131223.5 / 3C 207,0.680200,spec,08:40:47.58842599,+13:12:23.5637506,Radio,ICRF,J2000,-100.0,-100,-100,-99,-99.000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,671.0,3.000000e+00,21.189475,-99.0,Liu+2014 ; Krause+2019,2014ApJ...789..140L ; 2019MNRAS.482..240K,https://doi.org/10.1088/0004-637X/789/2/140 ; ...,Krause+ claim Radio Jet Precession,0.0,Binary SMBH Candidate,Binary SMBH Candidate,Binary SMBH Candidate,-99,-99,0.0,-99.0,-99.0,671.000000,Optical Fiber Spectroscopy / Optical LOS Radia...,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",Optical Spectroscopy / Fiber Optical Spectrosc...,Optical Fiber Spectroscopy / Optical LOS Radia...,-99,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",{}
2,3780,5433,5905,NaN,NaN,NaN,NaN,NaN,NaN,156.186667,19.205639,Table2,5433.0,Binary SMBH Candidate,-99,Optical Spectroscopy / Fiber Optical Spectrosc...,-99,102444.80+191220.3,0.827500,spec,10:24:44.80,+19:12:20.3,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,683.0,3.000000e+00,22.780073,-99.0,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140,,0.0,Binary SMBH Candidate,Binary SMBH Candidate,Binary SMBH Candidate,-99,-99,0.0,-99.0,-99.0,683.000000,Optical Fiber Spectroscopy / Optical LOS Radia...,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",Optical Spectroscopy / Fiber Optical Spectrosc...,Optical Fiber Spectroscopy / Optical LOS Radia...,-99,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",{}
3,1431,3074,3538,NaN,NaN,NaN,NaN,NaN,NaN,197.619432,32.345495,Table2,3074.0,Binary AGN Candidate,1308+326,Radio Jet Precession / Radio Imaging,-99,1308+326,0.995910,spec,13:10:28.6638,+32:20:43.783,Optical,SDSS,J2000,-99,-99,-99,-99,-99.000,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99.0,1.749798e-06,0.000014,-99.0,Britzen+2015 ; Britzen+2017,2015AN....336..471B ; 2017A&A...602A..29B,https://doi.org/10.1002/asna.201512178 ; https...,This article does not appear to have any rigor...,-0.5,Binary AGN Candidate,Binary AGN Candidate,Binary AGN Candidate,-99,-99,-0.5,-99.0,-99.0,-99.000000,Radio Imaging / Radio Jet Precession,"{'Radio': ['Imaging', 'Jet Precession']}",Radio Jet Precession / Radio Imaging / Radio I...,Radio Imaging / Radio Jet Precession,-99,"{

In [65]:
# here we're matching against the MAC targets that have two distinct sets of coordinates and the ICRF S/X band
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(crf_SX,MAC1,10)

print(len(idx1),len(idx2)) 
# 134 unique matches using 1'' tolerance 
# 135 unique matches using 3'' and 5'' tolerance
# 136 unique matches using 8'' and 10'' tolerance

tmatches



/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_72056/1350409438.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'


136 136


,level_0,index,Unnamed: 0,Designation,Flag,RA,DEC,RA_deg,Dec_deg,RA1_deg,Dec1_deg,Table_flag,Unnamed: 0.1,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Processed Selection Method,Parsed Selection Method,Analysis Method,Processed Analysis Method,Processed Confirmation Method,Parsed Analysis Method,Parsed Confirmation Method
0,3644,5297,5769,NaN,NaN,NaN,NaN,NaN,NaN,13.023167,0.593917,Table2,5297.0,Binary SMBH Candidate,-99,Optical Spectroscopy / Fiber Optical Spectrosc...,-99,005205.56+003538.1,0.399700,spec,00:52:05.56,+00:35:38.1,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.00000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,305.0,3.000000e+00,16.111440,-99.0,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140,,0.0,Binary SMBH Candidate,Binary SMBH Candidate,Binary SMBH Candidate,-99,-99,0.0,-99.0,-99.0,305.000000,Optical Fiber Spectroscopy / Optical LOS Radia...,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",Optical Spectroscopy / Fiber Optical Spectrosc...,Optical Fiber Spectroscopy / Optical LOS Radia...,-99,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",{}
1,1432,3075,3539,NaN,NaN,NaN,NaN,NaN,NaN,114.530807,17.705277,Table2,3075.0,Binary AGN Candidate,0735+178,Radio Jet Precession / Radio Imaging,-99,0735+178,0.450000,spec,07:38:07.39375312,+17:42:18.9981506,Radio,ICRF,J2000,-99,-99,-99,-99,-99.00000,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99.0,-9.900000e+01,-570.183843,-99.0,Britzen+2015,2015AN....336..471B,https://doi.org/10.1002/asna.201512178,This article does not appear to have any rigor...,-0.5,Binary AGN Candidate,Binary AGN Candidate,Binary AGN Candidate,-99,-99,-0.5,-99.0,-99.0,-99.000000,Radio Imaging / Radio Jet Precession,"{'Radio': ['Imaging', 'Jet Precession']}",Radio Jet Precession / Radio Imaging / Radio I...,Radio Imaging / Radio Jet Precession,-99,"{'Radio': ['Imaging', 'Jet Precession']}",{}
2,3669,5322,5794,NaN,NaN,NaN,NaN,NaN,NaN,116.423583,31.715722,Table2,5322.0,Binary SMBH Candidate,-99,Optical Spectroscopy / Fiber Optical Spectrosc...,-99,074541.66+314256.6,0.461600,spec,07:45:41.66,+31:42:56.6,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.00000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,-775.0,3.000000e+00,17.528909,-99.0,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140,Liu+2014 examined follow-up second-epoch spe...,0.0,Binary SMBH Candidate,Binary SMBH Candidate,Binary SMBH Candidate,-99,-99,0.0,-99.0,-99.0,-775.000000,Optical Fiber Spectroscopy / Optical LOS Radia...,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",Optical Spectroscopy / Fiber Optical Spectrosc...,Optical Fiber Spectroscopy / Optical LOS Radia...,-99,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",{}
3,3688,5341,5813,NaN,NaN,NaN,NaN,NaN,NaN,122.745792,41.567417,Table2,5341.0,Binary SMBH Candidate,-99,Optical Spectroscopy / Fiber Optical Spectrosc...,-99,081058.99+413402.7,0.506500,spec,08:10:58.99,+41:34:02.7,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.00000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,862.0,3.000000e+00,18.438597,-99.0,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140,,0.0,Binary SMBH Candidate,Binary SMBH Candidate,Binary SMBH Candidate,-99,-99,0.0,-99.0,-99.0,862.000000,Optical Fiber Spectroscopy / Optical LOS Radia...,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",Optical Spectroscopy / Fiber Optical Spectrosc...,Optical Fiber Spectroscopy / Optical LOS Radia...,-99,"{'Optical': ['Fiber Spectroscopy', 'LOS Radial...",{}
4,3695,5348,5820,NaN,